## Prepare Package and Data

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [55]:
import data
data.update_data()

/home/users/u5722780406/miniconda3/envs/tf/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2808: DtypeWarning: Columns (0,7) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/home/users/u5722780406/projects/sentimentModel/notebook/data.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  dfScore['service'][dfScore['service']==True] = 'TRUE'
/home/users/u5722780406/projects/sentimentModel/notebook/data.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  scored['class'] = scored['score'].apply(giveClass)


Creating the tf-idf matrix...

tf-idf features created
Creating the bag of words...

Done!


In [2]:
df = pd.read_csv("../dataset/facebook_comment_cleaned.tsv",sep='\t',encoding='utf-8')
df['token_arr'] = df['token'].apply(lambda x:eval(x))
scored = df[df['scoredAmt']>0]
scored['wordNum']= scored['token_arr'].apply(lambda x: len(x))

/home/users/u5722780406/miniconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [3]:
from gensim import models
model = models.Word2Vec.load('../model/fbcomment.w2v')

Using TensorFlow backend.


In [4]:
def giveClass(score):
    if(score==-32 ):
        return -32
    if(score==0):return 0
    if(score>0):return 1
    if(score<0):return -1
scored['class'] = scored['score'].apply(giveClass)
y = scored['class']
Y = pd.get_dummies(y)

/home/users/u5722780406/miniconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [65]:
#Let's use 15 as max_seq_len
max_seq_len = 15
k = 800
lessThanMaxSeqLen = scored['wordNum']<=max_seq_len
scoreddf = scored[lessThanMaxSeqLen]
X = scoreddf['token_arr'].values
Y = Y[lessThanMaxSeqLen]

/home/users/u5722780406/miniconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


In [6]:
X_train,X_test , y_train,y_test = train_test_split(X,Y.values,test_size=0.2,random_state=42)
X_test,X_valid , y_test,y_valid = train_test_split(X_test,y_test,test_size=0.5,random_state=42)
print("Train set Row: %d"%(X_train.shape[0]))
print("Test set Row: %d"%(X_test.shape[0]))
print("Validate set Row: %d"%(X_valid.shape[0]))

Train set Row: 4860
Test set Row: 608
Validate set Row: 608


## Create function to get a batch of data for model with padding

[สวัสดี ครับ กำลัง จะ ไป]  -> [PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD สวัสดี ครับ กำลัง จะ ไป]

In [64]:
mapping = [-32,-1,0,1]
def generate_batch(i,batch_size=32,verbose=False,X_train=X_train,y_train=y_train):
    comments = X_train[i:i+batch_size] 
    avaiRow = comments.shape[0]
    batch_X = np.zeros([batch_size,max_seq_len,k+2])
    batch_Y = y_train[i:i+batch_size]
    for i,comment in enumerate(comments):
        for j in range(15):
            if(max_seq_len-j>len(comment)): # Should not be word yet
                if(verbose):print("PAD",end=' ')
                batch_X[i][j][-1] = 1
            else: 
                if(verbose):print(comment[j-(max_seq_len-len(comment))],end=' ')
                try:
                    batch_X[i][j] = np.concatenate((model[comment[j-(max_seq_len-len(comment))]],np.zeros([2])))
                except Exception:
                    if(verbose):print("UNK",end=' ')
                    batch_X[i][j] = np.concatenate((np.zeros([k+1]),[1]))
        if(verbose):print(mapping[np.argmax(batch_Y[i])])
    if(verbose):print(i)
    return (batch_X[:avaiRow],batch_Y[:avaiRow])

import random
(batch_X,batch_Y) = generate_batch(random.randint(0,X_train.shape[0]),verbose=True)

PAD PAD PAD PAD PAD PAD PAD เนตเหี้ย มาก   เล่น เหี้ยไร ไม่ ได้ เลย 0
PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD 6 -1
PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD ผักบุ้ง ชุมนุม UNK รอบ กอง ไฟ -1
PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD รัก ลูก ทุก คน จ้า -1
PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD ขอบคุณ นะ คะ 1
PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD ราคา รวม ทั้งหมด 24110 UNK บาท 1
PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD 25 -1
PAD PAD PAD PAD PAD PAD PAD PAD จะ มี โปรนี้ อีก หรือ เปล่า ครับ 

IndexError: list index out of range

In [41]:
#Prepare test and validate set
(test_X,test_Y) = generate_batch(0,10000,X_train=X_test,y_train=y_test)
(valid_X,valid_Y) = generate_batch(0,10000,X_train=X_valid,y_train=y_valid)
test_X.shape

(608, 15, 802)

## Create model

Credit: http://monik.in/a-noobs-guide-to-implementing-rnn-lstm-using-tensorflow/

In [8]:
import tensorflow as tf

In [9]:
#Define two variables which will hols the input data and the target data.
data = tf.placeholder(tf.float32, [None, max_seq_len,k+2])
target = tf.placeholder(tf.float32, [None, 4])

In [10]:
#Create RNN cell
num_hidden = 24
cell = tf.nn.rnn_cell.LSTMCell(num_hidden,state_is_tuple=True)

In [11]:
val, state = tf.nn.dynamic_rnn(cell, data, dtype=tf.float32)

In [12]:
val = tf.transpose(val,[1,0,2])
last = tf.gather(val,int(val.get_shape()[0])-1)

In [13]:
weight = tf.Variable(tf.truncated_normal([num_hidden, int(target.get_shape()[1])]))
bias = tf.Variable(tf.constant(0.1, shape=[target.get_shape()[1]]))
prediction = tf.nn.softmax(tf.matmul(last,weight)+bias)

In [14]:
cross_entropy = -tf.reduce_sum(target * tf.log(tf.clip_by_value(prediction,1e-10,1.0)))

In [15]:
optimizer = tf.train.AdamOptimizer()
minimize = optimizer.minimize(cross_entropy)

/home/users/u5722780406/miniconda3/envs/tf/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [16]:
mistakes = tf.not_equal(tf.argmax(target, 1), tf.argmax(prediction, 1))
error = tf.reduce_mean(tf.cast(mistakes, tf.float32))

In [44]:
predictClass = tf.argmax(prediction, 1)
correct = tf.equal(tf.argmax(target, 1), tf.argmax(prediction, 1))
acc = tf.reduce_mean(tf.cast(correct, tf.float32))

In [17]:
init_op = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init_op)

In [ ]:
batch_size = 32
no_of_batches = int(len(X_train)/batch_size)
epoch = 5000
for i in range(epoch):
    ptr = 0
    avgAcc = 0
    for j in range(no_of_batches):
        #inp, out = train_input[ptr:ptr+batch_size], train_output[ptr:ptr+batch_size]
        (batch_X,batch_Y) = generate_batch(ptr)
        ptr+=batch_size
        sess.run(minimize,{data: batch_X, target: batch_Y})
        accuracy = sess.run(acc,{data:test_X,target:test_Y})
        avgAcc += accuracy/no_of_batches
        print(".",end='')
    print("Epoch - ",str(i))
    print("Avg acc: {}".format(round(avgAcc,2)))
    print("Show test result")
    selInt = random.randint(0,test_X.shape[0])
    test_pred = sess.run(predictClass,{data:test_X[selInt:selInt+10],target:test_Y[selInt:selInt+10]})
    test_acc = sess.run(acc,{data:test_X[selInt:selInt+10],target:test_Y[selInt:selInt+10]})
    for i,pred in enumerate(test_pred):
        
        print(X_test[selInt+i],end=' ')
        print(mapping[pred],end=" Ans: ")
        print(mapping[np.argmax(y_test[selInt+i])])
    print("Test accuracy: {}".format(round(test_acc,2)))
incorrect = sess.run(error,{data: test_input, target: test_output})
print('Epoch {:2d} error {:3.1f}%'.format(i + 1, 100 * incorrect))
sess.close()

.......................................................................................................................................................Epoch -  0
Avg acc: 0.62
Show test result
['วงกลม'] -32 Ans: -32
['ไม่', 'อิ่ม', 'ๆ'] -32 Ans: -32
['งาน', 'นี้', 'ต้อง', 'ใช้', 'บัตร', 'เข้า', 'ด้วย', 'หรอ', 'ครับ'] 1 Ans: 0
['กาก', 'ๆ'] -1 Ans: -1
['เครื่อง', 'ไม่', 'ได้', 'เรื่อง'] -32 Ans: -1
['Dtac', ' ', 'ครับ', ' ', 'ขอ', 'โปรโทร', 'เสาร์', 'อาทิตย์', ' ', '100', '.-', ' ', 'หน่อย', 'ครับ'] 0 Ans: 1
['6'] -32 Ans: -32
['หมุน', 'อยู่', 'นั่น', 'แหละ', 'ติ้ว', ' ', 'ๆ', 'ค่ะ'] -1 Ans: -1
['รอ', 'ช่าง', 'มา', 'ติด', 'เน็ต', 'จะ', 'เปน', 'เดือน', 'ล่ะ', 'ค่ะ'] 0 Ans: 0
['9'] -32 Ans: -32
Test accuracy: 0.699999988079071
.......................................................................................................................................................Epoch -  1
Avg acc: 0.62
Show test result
['ขอ', 'ทราบ', 'รายละเอียด', 'หน่อย', 'ได้', 'ไหม', 'คะ'] 1 Ans: -32
['หา'

.......................................................................................................................................................Epoch -  10
Avg acc: 0.62
Show test result
['แก้ใข', 'เสร็จ', 'เมื่อ', 'ไหร่'] -32 Ans: -1
['Oppo', ' ', 'r9s', ' ', 'plus', ' ', 'รองรับ', ' ', '3', 'ca', ' ', 'มั้ย', 'ครับ'] 1 Ans: 0
['ถ้า', 'แบต', 'เสื่อม', 'มี', 'จำหน่าย', 'เป่า'] -32 Ans: 1
['เบอร์', '098717641', '6', '.', ' ', 'ช่วย', 'ตอบ', 'ด้วย', 'ครับ'] 1 Ans: 0
['ขอ', 'เป็น', 'เน็ต', '20', 'GB', 'ได้', 'ไหม', '555'] 1 Ans: 1
['สนใจ', 'ทำไง', 'ค่ะ'] -32 Ans: 1
['อ่าน', 'ภาษา', 'อังกิด', 'ไม่', 'ค่อย', 'ออก', 'เเปล', 'หน่อย', 'จร้า'] -32 Ans: 0
['ใคร', 'ซื้อ', 'ได้', 'บ้าง', 'ค่ะ'] 0 Ans: 0
['ตอบ', 'หน่อย'] 0 Ans: -32
['ต่าง', 'จังหวัด', 'ไม่', 'มี', 'หรอ', 'ค่ะ'] 0 Ans: -32
Test accuracy: 0.20000000298023224
.......................................................................................................................................................Epoch -  11
Avg acc:

.......................................................................................................................................................Epoch -  20
Avg acc: 0.62
Show test result
['66'] -32 Ans: -32
['คุณ', 'เพชรลดา', 'ครับ', 'แจ้ง', 'คุณชมณัส', 'หรือ', 'ยัง', 'ครับ'] 0 Ans: -32
['ถ้า', 'หมด', ' ', '10', ' ', 'GB', ' ', 'FUP', ' ', 'เท่า', 'ไหร่', 'ครับ', 'เนี่ย'] 0 Ans: 1
['เครือค่าย', 'กาก', 'มึง', 'จะ', 'ลด', 'สปีด', 'เน็ด', 'กุหา', 'พ่อ', 'มึง', 'ติ'] -1 Ans: -1
['คอนโดชั้น', '23', ' ', 'ยัง', 'ไม่', 'รอด', '....', 'เหอ', 'ๆ'] -32 Ans: -32
['ยอด', 'มาก', 'ๆ'] 0 Ans: 0
['ย้าย', 'ค่าย', 'มา', 'แม่งกาก', 'สัส'] -1 Ans: -1
['อ้อนวอน', 'คน', 'ไทย', '?', ' ', 'นึก', 'ว่า', 'มี', 'โปร', 'ใหม่', 'ๆ'] -32 Ans: -32
['#', 'เมื่อ', 'วาน', 'เชียงใหม่', 'ก็', 'สะดุด', 'ครับ', ' ', '16gb', ' ', 'ล่มสลาย'] 1 Ans: -1
['នេះកូនសំលាាញ់'] -32 Ans: -32
Test accuracy: 0.699999988079071
.........................................................................................................

.......................................................................................................................................................Epoch -  30
Avg acc: 0.62
Show test result
['ขอ', 'เป็น', 'เน็ต', '20', 'GB', 'ได้', 'ไหม', '555'] 0 Ans: 1
['สนใจ', 'ทำไง', 'ค่ะ'] -32 Ans: 1
['อ่าน', 'ภาษา', 'อังกิด', 'ไม่', 'ค่อย', 'ออก', 'เเปล', 'หน่อย', 'จร้า'] -32 Ans: 0
['ใคร', 'ซื้อ', 'ได้', 'บ้าง', 'ค่ะ'] 0 Ans: 0
['ตอบ', 'หน่อย'] 0 Ans: -32
['ต่าง', 'จังหวัด', 'ไม่', 'มี', 'หรอ', 'ค่ะ'] 0 Ans: -32
['16'] -32 Ans: -32
['my', ' ', 'cat', ' ', 'มี', 'ก่อน', 'ตั้ง', 'นาน', 'แล้ว'] 0 Ans: 1
['เนตช้า', 'มาก', 'โปร', '599'] 1 Ans: -1
['แล้ว', 'ที่', 'ผ่าน', 'มา', ' ', '2', ' ', 'เดือน', ' ', 'มัน', 'คือ', 'อะไร'] 0 Ans: -32
Test accuracy: 0.20000000298023224
.......................................................................................................................................................Epoch -  31
Avg acc: 0.62
Show test result
['ปรับปรุง', 'เน็น', 'หน่อย', ' ', 

.......................................................................................................................................................Epoch -  40
Avg acc: 0.62
Show test result
['J7', ' ', 'prime', 'เท่า', 'รัย'] 1 Ans: 1
['กลโกง', 'ที่', 'แยบยล', 'มาก', 'ค่ะ'] 0 Ans: -32
['ทำไม', 'ไม่', 'เป็น', ' ', '4', 'Gครับ'] -1 Ans: 0
['อย่าก', 'ได้', 'บาเรตpb', 'มี', 'ปะ'] 0 Ans: -32
['Ungkana', ' ', 'Tanpragoon'] -32 Ans: -32
['เห้อ', 'อออ', '.', ' ', ' ', 'มี', 'เงื่อนไข', '.', ' ', 'เซ็ง', 'ชมัด'] -1 Ans: 0
['ทรู', 'ไวไฟ', ' ', 'กาก', 'มาก', ' ', 'เข้า', 'ไม่', 'ได้', 'เลย', 'เกิด', 'ไร', 'ขึ้น'] -1 Ans: -1
['อยาก', 'ได้', 'สัก', '59', 'บ', '.', '/', '512', '/', '7', 'วัน', 'ไม่', 'ลด', 'สปีด'] -1 Ans: 0
['75'] -32 Ans: -32
['0'] -32 Ans: -32
Test accuracy: 0.5
.......................................................................................................................................................Epoch -  41
Avg acc: 0.63
Show test result
['ผ่อน', 'กับ', 'บัตร', 

...

.......................................................................................................................................................Epoch -  110
Avg acc: 0.62
Show test result
['อยาก', 'ได้', 'เครื่อง', 'ไม่', 'อยาก', 'ได้', 'โบนัส'] -32 Ans: 0
['@', '.', 'truewifi ', 'ทำไม', 'ใช้', 'ไม่', 'ได้', 'ครับ', 'ผม', 'อยู่', 'เพชร', 'เกษม', '112', '.'] 0 Ans: -1
['สนใจ', 'ค่ะ'] 1 Ans: 1
['ิอยาก', 'ได้j7'] -32 Ans: 1
['Nut', ' ', 'Jung', ' ', 'Ruy'] -32 Ans: -32
['กระบองเพชร'] -32 Ans: -32
['ตาม', 'ที่', 'พูด', 'กัน'] -1 Ans: -32
['3', ',', '000'] -32 Ans: -32
['4', ' ', 'ค่ะ'] -32 Ans: -32
['แย่', 'ถาม', 'ไร', 'ไม่', 'ค่อย', 'ตอบ', 'แย่', 'ๆ', 'ๆ'] -1 Ans: -1
Test accuracy: 0.6000000238418579
.......................................................................................................................................................Epoch -  111
Avg acc: 0.62
Show test result
['ย้าย', 'ค่าย', 'หนี', 'เลย', 'ถ้า', 'ไม่', 'ดี'] -1 Ans: -1
['are', ' ', 'you', ' ', 'a']

.......................................................................................................................................................Epoch -  120
Avg acc: 0.62
Show test result
['جميل', ' ', 'جدا', ' ', 'من', ' ', 'الفريق', ' ', 'TRUE', ' ', ' ', 'move', ' ', ' ', 'h'] -32 Ans: -32
['มา', 'ด่า', 'ละสิ'] -32 Ans: -32
['สมัคร', 'เเบบ', 'ราย', 'เดื่อน', 'เสีย', 'เท่า', 'ไร่', 'ตับ'] -32 Ans: 0
['ขอ', 'ร่วม', 'สนุก', 'ตอบ', 'คำ', 'ถาม', 'ด้วย', 'คน', 'นะ', 'ครับ'] 0 Ans: -32
['AIS', ' ', 'สุด', 'ยอด', 'แฮง'] 1 Ans: 0
['Siwagon', ' ', 'Suvipanon'] -32 Ans: -32
['ช้า', 'ไป', 'ไหม', ' \n', 'แต่', 'ขอบคุณ', 'ที่', 'มี', 'รถ', 'ไป', 'ให้', 'สัญญาน', 'ที่', 'สนามหลวง'] -1 Ans: 0
['แคนาดา'] -32 Ans: 0
['Sirawich', ' ', 'Voungchuy', 'เปลี่ยน', 'มา', 'ใช้', 'โปรนี้', 'กัน'] -32 Ans: 0
['ขึ้น', 'H+', 'พอดู', 'ไม่', 'มี', 'สัญญาณ', 'นี้', 'ใช่', 'ทรู', 'มา', 'โพส', 'เลย', 'นะ', 'หงุดหงิด', 'เว้ย'] -1 Ans: -1
Test accuracy: 0.4000000059604645
.........................................

.......................................................................................................................................................Epoch -  130
Avg acc: 0.62
Show test result
['ประเทศ', 'ไทย', '..', 'มีนาเยอะ'] 0 Ans: -32
['แล้ว', 'เครื่อง', 'ที่', 'ซื้อ', 'มา', 'มี', 'ปัญหา', 'ตรง', 'ที่', 'ใส่', 'เม็ม', 'ทำไง', 'ค่ะ'] 1 Ans: 1
['ส'] -32 Ans: -32
['Preecha', ' ', 'Mantra'] -32 Ans: -32
['ถ้า', 'จอ', '6', '"', 'ก็', 'ว่า', 'ไป', 'ครับ'] 1 Ans: 0
['เก่ง', 'แต่', 'พูด', 'ไป', 'วัน', 'ๆ'] -1 Ans: -32
['เรื่อง', 'จริง'] -32 Ans: -32
['ใน', 'ภาพ', 'นี้', 'สี', 'คิ้ว', 'นี่'] -32 Ans: -32
['Sunisa', ' ', 'Leksung'] -32 Ans: -32
['Petcharak S.', ' ', 'Cheyjanpli'] -32 Ans: -32
Test accuracy: 0.699999988079071
.......................................................................................................................................................Epoch -  131
Avg acc: 0.61
Show test result
['หมด', 'เขต', 'ตอน', 'ไหน'] 0 Ans: 0
[',', 'อยาก', 'ไดิทีื', '890', 'คั่

.......................................................................................................................................................Epoch -  150
Avg acc: 0.61
Show test result
['66'] -32 Ans: -32
['คุณ', 'เพชรลดา', 'ครับ', 'แจ้ง', 'คุณชมณัส', 'หรือ', 'ยัง', 'ครับ'] 0 Ans: -32
['ถ้า', 'หมด', ' ', '10', ' ', 'GB', ' ', 'FUP', ' ', 'เท่า', 'ไหร่', 'ครับ', 'เนี่ย'] 0 Ans: 1
['เครือค่าย', 'กาก', 'มึง', 'จะ', 'ลด', 'สปีด', 'เน็ด', 'กุหา', 'พ่อ', 'มึง', 'ติ'] -1 Ans: -1
['คอนโดชั้น', '23', ' ', 'ยัง', 'ไม่', 'รอด', '....', 'เหอ', 'ๆ'] -1 Ans: -32
['ยอด', 'มาก', 'ๆ'] 0 Ans: 0
['ย้าย', 'ค่าย', 'มา', 'แม่งกาก', 'สัส'] -1 Ans: -1
['อ้อนวอน', 'คน', 'ไทย', '?', ' ', 'นึก', 'ว่า', 'มี', 'โปร', 'ใหม่', 'ๆ'] -32 Ans: -32
['#', 'เมื่อ', 'วาน', 'เชียงใหม่', 'ก็', 'สะดุด', 'ครับ', ' ', '16gb', ' ', 'ล่มสลาย'] 1 Ans: -1
['នេះកូនសំលាាញ់'] -32 Ans: -32
Test accuracy: 0.6000000238418579
........................................................................................................

.......................................................................................................................................................Epoch -  160
Avg acc: 0.61
Show test result
['ไป', 'ที่', 'เค้าท์เตอร์', 'จะ', 'มี', 'โปรนี้', 'อยู่', 'ไม', 'ค่ะ'] 1 Ans: 0
['Sukanda', ' Saetang', ' ', 'Bua ', 'Sookgont', ' ', 'อัน', 'นี้', 'อ่ะ'] -32 Ans: -32
['เติม', 'เท่า', 'ไหร่', 'หรอ', '?', '?', '?'] 1 Ans: -32
['ปลา', 'ไหล', 'ผัด', 'เผ็ด', 'มั่ง', '555'] -32 Ans: -32
['อยาก', 'ได้', 'ตุ๊กตา', 'ค่ะ'] -32 Ans: -32
['ยกเลิก', 'เน็ต', 'ทำ', 'ยัง', 'ไง', 'คัปปป'] -1 Ans: 0
['ไม่', 'มี', 'บัตร', 'เครดิต', 'ผ่อน', 'ได้', 'ไหม', 'คับ'] 0 Ans: 1
['จระเข้', 'พิการ'] -32 Ans: -32
['Yuki', ' ', 'Tigeryu', ' ', 'รัก', 'ในหลวง'] -32 Ans: -32
['เครื่อง', 'เปล่า', 'ราคา', 'เท่า', 'ไหร่', 'ครับ'] 0 Ans: 1
Test accuracy: 0.5
.......................................................................................................................................................Epoch -  161
Avg acc: 

(608, 15, 802)